In [ ]:
%matplotlib notebook
from scipy.special import binom
from scipy.stats import triang
from scipy.stats import beta

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider
import ipywidgets as widgets
from IPython.display import display
sns.set()

# Bayes'sche Statistik

## Bayes am Beispiel des Münzwurfs

Auf der Basis von:
Kruschke, _Doing Bayesian Analysis_ , 2nd Edition, Elsevier Inc. 2015

### Münzwurfsimulation

Anteil von Kopf (_Head_ bzw. _H_) unter der gegeben Anzahl von Münzwürfen:

In [ ]:
def coin_flip(n,p):
    seq = np.random.binomial(1, p, n)
    seq_sym = ['H' if c == 1 else 'T' for c in seq]
    head_freq = seq.cumsum() / np.arange(1,n+1)
    return seq, seq_sym, head_freq

def plot_coin_flips(n,p):
    plt.figure(figsize=(10,5))
    plt.grid(True)
    _,flips, head_freqs = coin_flip(n,p)
    print(''.join(flips))
    print(f'Anteil von Kopf/H an allen Würfen {head_freqs[-1]}')
    plt.plot(np.arange(1,n+1),head_freqs)
    plt.xlabel("Anzahl Würfe")
    plt.ylabel("Anteil Kopf bzw. Head")
    plt.show()

#np.random.seed(1)
interact(plot_coin_flips, 
         n=IntSlider(min=1, max=1000, value=10, continuous_update=False), 
         p=FloatSlider(min=0.0, max=1.0, value=0.5, continuous_update=False));

### Dreiecksverteilung als Prior

In [ ]:
def coin_flip_likelihood(n,h,theta):
    return theta**h * (1-theta)**(n-h)

def plot_coin_flip_distribution(n=1, h=1, theta=0.5, resolution=101):
    theta_range = np.linspace(0.0,1.0,resolution)
    # Prior
    triang_prior = triang.pdf(theta_range, theta)
    triang_prior /= sum(triang_prior)
    # Likelihood
    likelihood = np.array([coin_flip_likelihood(n,h,th) for th in theta_range])
    # Posterior
    posterior = np.multiply(likelihood,triang_prior)
    posterior /= sum(posterior)
    
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(9, 10), sharex=True)
    ax1, ax2, ax3 = axes
    ax1.plot(theta_range, triang_prior)
    ax1.grid(True)
    ax1.set_title("Prior")
    ax1.set_ylabel(r'$p(\theta)$')
    ax2.plot(theta_range, likelihood)
    ax2.grid(True)
    ax2.set_title("Likelihood")
    ax2.set_ylabel(r'$p(D|\theta)$')
    ax3.plot(theta_range, posterior)
    ax3.grid(True)
    ax3.set_title("Posterior")
    ax3.set_ylabel(r'$p(\theta|D)$')
    ax3.set_xlabel(r'$\theta$')
    plt.show()

plot_coin_flip_distribution(1,0) # Ein Münzwurf -> H

In [ ]:
# Define sliders for n and h:
n_slider = IntSlider(min=1, max=50, value=10, continuous_update=False)
h_slider = IntSlider(min=0, max=10, value=0, continuous_update=False)

def update_h_range(*args):
    h_slider.max = n_slider.value
n_slider.observe(update_h_range, 'value')

# Define widget for the Coin-Flip:
coin_flip_widget = interactive(plot_coin_flip_distribution,
                                      n = n_slider,
                                      h = h_slider,
                                      theta = FloatSlider(min=.1, max=.9, value=0.5, continuous_update=False),
                                      resolution = IntSlider(min=50, max=201, value=101, continuous_update=False))
display(coin_flip_widget)

Betrachtet man die Posterior-Wahrscheinlichkeit der Teilsequenz einer Wurffolge der Länge $n$ für eine gegebene Münze (Wahrscheinlichkeit für das Werfen eines Kopfes $p$ gegeben):

In [ ]:
def plot_coin_flip_sequence_distribution(n=1, theta=0.5, resolution=101, only_last_likelihood=True):
    flips,flips_sym, head_freqs = coin_flip(n,theta)
    print(''.join(flips_sym))
    print(f'Anteil von Kopf/H an allen Würfen {head_freqs[-1]}')
    
    theta_range = np.linspace(0.0,1.0,resolution)
    # Prior
    triang_prior = triang.pdf(theta_range, theta)
    triang_prior /= sum(triang_prior)
    
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(9, 10), sharex=True)
    ax1, ax2, ax3 = axes
    ax1.plot(theta_range, triang_prior)
    ax1.grid(True)
    ax1.set_title("Prior")
    ax1.set_ylabel(r'$p(\theta)$')
    
    ax2.grid(True)
    ax2.set_title("Likelihood of last Iteration")
    #ax2.set_ylim(0.0,0.4)
    ax2.set_ylabel(r'$p(D|\theta)$')
    
    ax3.grid(True)
    ax3.set_title("Posterior")
    ax3.set_ylabel(r'$p(\theta|D)$')
    ax3.set_xlabel(r'$\theta$')
    
    for i in range(n):
        heads = sum(flips[:i+1])
        # Likelihood
        likelihood = np.array([coin_flip_likelihood(i+1,heads,th) for th in theta_range])
        if not(only_last_likelihood):
            ax2.plot(theta_range, likelihood, color='b', alpha=max(.2, float(i+1)/n))
        elif i == n - 1:
            ax2.plot(theta_range, likelihood, color='b')
        # Posterior
        posterior = np.multiply(likelihood,triang_prior)
        posterior /= np.sum(posterior)
        ax3.plot(theta_range, posterior, color='b', alpha=max(.2, float(i+1)/n))
    
    plt.show()

#np.random.seed(1)    
interact(plot_coin_flip_sequence_distribution,
         n = IntSlider(min = 1, max = 200, value = 10, continuous_update = False),
         theta = FloatSlider(min = 0.0, max = 1.0, value = .5, continous_update = False),
         resolution = IntSlider(min = 81, max = 301, value = 101, continuous_update = False),
         only_last_likelihood=True)

### Die Beta-Verteilung

Die _Beta-Verteilung_ wird durch die Parameter _a_ und _b_ bestimmt. Man kann sich die Anzahl an geworfenen _Heads_ (_H_) als _a_ und die Anzahl an geworfenen _Tails_ (_T_) als _b_ vorstellen. 

In [ ]:
x = np.linspace(0, 1, 200)
def plot_beta_distribution(a=1, b=1):
    plt.figure()
    plt.grid(True)
    plt.plot(x, beta.pdf(x, a, b))
    
interact(plot_beta_distribution, 
         a=IntSlider(min=1, max=50, value=3, continuous_update=False), 
         b=IntSlider(min=1, max=50, value=10, continuous_update=False));

Wird die Beta-Verteilung als Prior für die Münze benutzt und die Münze $n$ mal geworfen:

In [ ]:
%precision 4

def plot_coin_flip_sequence_beta_prior(a=1, b=1, n=10, theta=0.5, resolution=101, update_prior_with_previous_posterior = True, only_last_likelihood=True):
    flips,flips_sym, head_freqs = coin_flip(n,theta)
    
    theta_range = np.linspace(0.0,1.0,resolution)
    # Prior
    beta_prior = beta.pdf(theta_range, a, b)
    beta_prior /= sum(beta_prior)
    
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(9, 10), sharex=True)
    ax1, ax2, ax3 = axes
    ax1.plot(theta_range, beta_prior)
    ax1.grid(True)
    ax1.set_title("Prior")
    ax1.set_ylabel(r'$p(\theta)$')
    
    ax2.grid(True)
    ax2.set_title("Likelihood of last Iteration")
    #ax2.set_ylim(0.0,0.4)
    ax2.set_ylabel(r'$p(D|\theta)$')
    
    ax3.grid(True)
    ax3.set_title("Posterior")
    ax3.set_ylabel(r'$p(\theta|D)$')
    ax3.set_xlabel(r'$\theta$')
    
    for i in range(n):
        heads = sum(flips[:i+1])
        # Likelihood
        likelihood = np.array([coin_flip_likelihood(i+1,heads,th) for th in theta_range])
        if not(only_last_likelihood):
            ax2.plot(theta_range, likelihood, color='b', alpha=max(.2, float(i+1)/n))
        elif i == n - 1:
            ax2.plot(theta_range, likelihood, color='b')
        # Posterior
        posterior = np.multiply(likelihood,beta_prior)
        posterior /= np.sum(posterior)
        ax3.plot(theta_range, posterior, color='b', alpha=max(.2, float(i+1)/n))
        if update_prior_with_previous_posterior:
                beta_prior[:] = posterior
        
    print(''.join(flips_sym))
    print(f'Anteil von Kopf/H an allen Würfen {head_freqs[-1]:.4f}')
    print(f'Estimatation of p (Posterior Mean): {np.dot(theta_range,posterior):.4f}')
    print(f'Posterior Mode: {theta_range[np.argmax(posterior)]:.4f}')
    plt.show()

#np.random.seed(1)    
interact(plot_coin_flip_sequence_beta_prior,
         a=IntSlider(min=1, max=50, value=10, continuous_update=False), 
         b=IntSlider(min=1, max=50, value=10, continuous_update=False),
         n = IntSlider(min = 1, max = 200, value = 10, continuous_update = False),
         theta = FloatSlider(min = 0.0, max = 1.0, value = .5, continous_update = False),
         resolution = IntSlider(min = 81, max = 301, value = 101, continuous_update = False),
         update_prior_with_previous_posterior=True,
         only_last_likelihood=True)